In [19]:
import numpy as np
from sklearn.manifold import TSNE
from pathlib import Path
import xarray as xr
from hmpai.normalization import *
from hmpai.data import preprocess
from hmpai.training import split_data_on_participants, train_and_evaluate, k_fold_cross_validate, get_compile_kwargs
import matplotlib.pyplot as plt
from hmpai.utilities import MASKING_VALUE, set_global_seed
from sklearn.linear_model import LogisticRegression

In [27]:
data_path = Path("../data/sat1/split_stage_data.nc")
data = xr.load_dataset(data_path)
set_global_seed(42)
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)
train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [28]:
# stage_durations = np.isnandata.where(data.data != MASKING_VALUE)
# np.isnan(test_set.data.where(test_set.data != MASKING_VALUE))

train_lengths = np.isnan(train_data.data.where(train_data.data != MASKING_VALUE)).argmax(
        dim=["samples", "channels"]
    )['samples']

test_lengths = np.isnan(test_data.data.where(test_data.data != MASKING_VALUE)).argmax(
        dim=["samples", "channels"]
    )['samples']

train_lengths_data = train_lengths.data.reshape(-1, 1)
test_lengths_data = test_lengths.data.reshape(-1, 1)

In [33]:
log_res = LogisticRegression(random_state=42, class_weight='balanced').fit(train_lengths_data, train_lengths.labels)

In [34]:
log_res.score(test_lengths_data, test_lengths.labels)

0.4382360310699073